In [1]:
!pip install transformers
!pip install datasets 
!pip install sentencepiece
!pip install rouge
! python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.8 MB/s 
     |████████████████████████████████| 7.6 MB 45.5 MB/s 
     |████████████████████████████████| 163 kB 51.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 6.3 MB/s 
     |████████████████████████████████| 115 kB 53.2 MB/s 
     |████████████████████████████████| 212 kB 53.7 MB/s 
     |████████████████████████████████| 127 kB 45.3 MB/s 
     |████████████████████████████████| 115 kB 48.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 6.9 MB/s 
Looking in indexes: https

In [2]:
import torch
import pandas as pd
import numpy as np
import datasets
import torch.nn as nn
import transformers
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
import spacy
import string

from datasets import load_dataset, load_metric

dataset = load_dataset("multi_news",split = 'train')

Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Dataset multi_news downloaded and prepared to /root/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72. Subsequent calls will reuse this data.


In [3]:
!pip install pysbd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 71 kB 4.4 MB/s 


In [4]:
nlp = spacy.load("en_core_web_sm")
train_doc, train_sum = dataset["document"][0], dataset["summary"][0]
doc = nlp(train_doc)
print(list(doc.sents))

[National Archives 
 
 , Yes, it’s that time again, folks., It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. 
 
 , A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m., New York time offering one of the most important snapshots on how the economy fared during the previous month., Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February., The unemployment rate is expected to hold steady at 8.3%. 
 
 , Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires., Feel free to weigh-in yourself, via the comments section., And while you’re here, why don’t you sign up to follow us on Twitter., 
 
 , Enjoy the show., ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may n

In [19]:
sen = [i.text for i in doc.sents]


National Archives 
 
 
Yes, it’s that time again, folks.
It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. 
 
 
A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m.
New York time offering one of the most important snapshots on how the economy fared during the previous month.
Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February.
The unemployment rate is expected to hold steady at 8.3%. 
 
 
Here at MarketBeat HQ, we’ll be offering color commentary before and after the data crosses the wires.
Feel free to weigh-in yourself, via the comments section.
And while you’re here, why don’t you sign up to follow us on Twitter.

 
 
Enjoy the show.
||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing

In [27]:
ent = [i.text for i in doc.ents]
document = []

for i in ent:
    for sen in doc.sents:
        if sen not in document:
            if str(i) in str(sen):
                document.append(sen)

document

[National Archives 
  
  ,
 It’s the first Friday of the month, when for one ever-so-brief moment the interests of Wall Street, Washington and Main Street are all aligned on one thing: Jobs. 
  
  ,
 New York time offering one of the most important snapshots on how the economy fared during the previous month.,
 Expectations are for 203,000 new jobs to be created, according to economists polled by Dow Jones Newswires, compared to 227,000 jobs added in February.,
 But Federal Reserve Chairman Ben Bernanke has cautioned that the current hiring pace is unlikely to continue without more consumer spending.,
 A fresh update on the U.S. employment situation for January hits the wires at 8:30 a.m.,
 ||||| Employers pulled back sharply on hiring last month, a reminder that the U.S. economy may not be growing fast enough to sustain robust job growth.,
 The unemployment rate fell to 8.2 percent, the lowest since January 2009.,
 The unemployment rate is expected to hold steady at 8.3%. 
  
  ,
 And

In [28]:
from spacy import displacy
spacy.displacy.render(doc, style='ent', jupyter=True)

In [30]:
dataset

Dataset({
    features: ['document', 'summary'],
    num_rows: 44972
})

In [46]:
import time

In [49]:
start_time = time.time()
document = []

for each in dataset["document"][0:1000]:
    d = []
    doc = nlp(each)
    ent = [i.text for i in doc.ents]
    

    for i in ent:
        for sen in doc.sents:
            if sen not in d:
                if str(i) in str(sen):
                    d.append(sen)
    document.append(d)
end_time = time.time()
print(end_time-start_time)

2880.7812502384186


In [50]:
len(dataset["document"])

44972